In [52]:
import os
import json
import requests
import pandas as pd
from sqlalchemy import create_engine, text
from dotenv import load_dotenv
from urllib.parse import quote_plus
from bs4 import BeautifulSoup, Comment, Doctype
import re
import time 


# Cargar variables de entorno (tu GOOGLE_API_KEY desde el archivo .env)
load_dotenv()

True

In [53]:
# Credenciales de la Base de Datos ANTIGUA 
DB_OLD_USER = "root"
DB_OLD_PASS = "" # Sin contraseña
DB_OLD_HOST = "127.0.0.1"
DB_OLD_NAME = "sai_v2"

# Credenciales de la Base de Datos NUEVA 
DB_NEW_USER = "root"
DB_NEW_PASS = "" # Sin contraseña
DB_NEW_HOST = "127.0.0.1"
DB_NEW_NAME = "sai_v5"

endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("OPENAI_API_KEY")

STATIC_CONTENT_STRUCTURE = {
    "en": {
        "special_tag": "Urgent Help",
        'description_heading'    : 'Description Heading',
        'description_text'       : 'Detailed Description',
        'summary_title'          : 'Summary Title',
        'summary_text'           : 'Summary Text',
        'help_mini_title'        : "Help People In need",
        'help_title'             : "Start Donating Today, Make The Difference",
        'help_btn_text'          : "DONATE TO THIS PROJECT >"
    },
    "es": {
        "special_tag": "Ayuda urgente",
        "description_heading": "Subtitulo por defecto",
        "description_text": "Texto de descripción por defecto",
        'summary_title': 'Titulo de resumen',
        "summary_text": "Texto de resumen",
        'help_mini_title': "Ayuda a personas en necesidad",
        'help_title': "Comienza a donar hoy, Haz la diferencia",
        'help_btn_text': "DONAR A ESTE PROYECTO >"
    }
}

STATIC_CONTENT_JSON = json.dumps(STATIC_CONTENT_STRUCTURE, ensure_ascii=False)

with open("promptproject2.txt", 'r', encoding='utf-8') as f:
    system_prompt = f.read()




In [1]:

def fetch_new_gallery_data():
    engine = create_db_engine(DB_NEW_USER, DB_NEW_PASS, DB_NEW_HOST, DB_NEW_NAME) ##creo un motor de busqueda
    query = "SELECT id,title FROM projects" ## selecciona las imagenes de pages, y que solo contenga campaing

    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df




def insert_photos(id,ext):
    engine_new = create_db_engine(DB_NEW_USER, DB_NEW_PASS, DB_NEW_HOST, DB_NEW_NAME) ##creo un motor de busqueda
    
    insert_query = text("""
        INSERT INTO uphoto (
            modelid,ext, created_at, updated_at
        ) VALUES (
            :modelid,:ext, NOW(), NOW()
        )
    """)
    try:
        with engine_new.connect() as connection:
            # Correctly define the 'record' dictionary with parameter values
            record_data = {
             
                "modelid": id,
                "ext": ext,
                
            }
            connection.execute(insert_query, record_data)
            connection.commit()
            return True # Indicate success
    except Exception as e:
        print(f"Error inserting photo: {e}")
        # Optionally, you could log the error here using a proper logging framework
        return False # Indicate failure
       



def fetch_old_gallery_data():
    engine = create_db_engine(DB_OLD_USER, DB_OLD_PASS, DB_OLD_HOST, DB_OLD_NAME) ##creo un motor de busqueda
    query = "SELECT gallery,title,id FROM pages WHERE layout='campaign'" ## selecciona las imagenes de pages, y que solo contenga campaing

    with engine.connect() as connection:
        df = pd.read_sql(query, connection)
    return df



def agregar_id(lista,query):




    lista2=[]

    size=len(lista)

    for i in range(0,size):
        title=lista[i][1]

        size_query=len(query)
        for j in range(0,size_query):
            title_aux=query.iloc[j,1]
            id=int(query.iloc[j,0])
            
            title_clean=limpiar_vector(title_aux)
            if(title_clean==title):
                lista[i].append(id)
    
    
                
  
       
        
   

def limpiar_vector(title):

    vectorAux=title.split(",") ##divido en una lista el string
    limpieza=vectorAux[0] ##selecciono el titulo en ingles
    envio=limpieza[8:-1] ##con esto quito la primera parte {"end"
    return envio
    


def convert_into_vector(d):


    ##primera columna sera id
    ## segundo el titulo antiguo
    ## tercero la imagen

    
 
    size=len(d)
    listaPrincipal=[]

    for i in range(0,size):
        ## entra en la lista de valores de imagenes

        if(d.iloc[i,0]!=None):

            ##enttra en un bucle para obtener mas informacion acerca de las imagenes y su id


            imagenes=vector_imagenes(d.iloc[i,0]) ##convierte el string a un vector de imagenes

            size2=len(imagenes) ##recorremos cada imagen en este punto del array

            for j in range(0,size2):
                imagen=imagenes[j]
                listaAux=[] ##cree esta lista auxiliar para meterle valores a esto
                id=int(d.iloc[i,2])
                listaAux.append(id) ##coloco el id de primero
                listaAux.append(d.iloc[i,1]) ##coloco el title de segundo
                listaAux.append(imagen) ##coloco la imagen
                listaPrincipal.append(listaAux) ##creacion de lista anidada

           ## for j in range(0,size2):
    
    return listaPrincipal



def vector_imagenes(d):
    imagenes=[]
    text_1=d[1:-1]##limpieza de textos
    vector_1=text_1.split(",") ##convierte a las imagenes en una lista a traves de la funcion split!! pendiente

    size=len(vector_1) ## obtiene tamano de las imagenes pal for que hare

    for i in range(0,size):
        elemento=vector_1[i] ##trabajando con elemento especifico
        vector_aux=elemento.split("/") ##cree un vector auxiliar para dividir lo que obtenia de imagenes
        imagen=vector_aux[2][:-1] ##con esto obtengo la extension de la imagen
        imagenes.append(imagen) ## agrego a la imagen pura loquera

    return imagenes

    





def create_db_engine(user, password, host, db_name):
    """Crea un motor de conexión SQLAlchemy."""
    # Codifica la contraseña si contiene caracteres especiales
    password_encoded = quote_plus(password)
    connection_string = f"mysql+mysqlconnector://{user}:{password_encoded}@{host}/{db_name}"
    return create_engine(connection_string)


#query1=fetch_old_gallery_data()
#lista=convert_into_vector(query1)
#query2=fetch_new_gallery_data()


#agregar_id(lista,query2)




#for i in range(0,len(lista)):
    ##insertar valores obtenidos
    #insert_photos(lista[i][3],lista[i][2])
    